# Basic ETL with Pandas, Azure Cosmos DB and GitHub Codespaces
Convert a filtered CSV File into JSON, then insert into Azure Cosmos DB in minutes with GitHub Codespaces

1. [Create a Cosmos DB NoSQL Account, you can stop after creating the resource](https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/quickstart-portal?WT.mc_id=academic-99884-jasmineg)

2. After account is created in the Azure Portal, navigate to the resource (you can find it in your notifications)

3. To the right of the resource overview select `Keys`. locate the `URI`, and `PRIMARY KEY` secrets.

    ![Azure cosmos db secrets](img/azcosmosdb_secrets.png)
 
4. Set copied secrets it as secrets in your [Codespaces settings here](https://github.com/settings/codespaces). 
    **`URI` should be the `COSMOS_ENDPOINT` secret and `PRIMARY KEY` should be the  `COSMOS_KEY` secret**
     ![codespaces secrets](img/codespaces_secret_settings.png)

5. Run this Notebook

6. **[Clean up your Cosmos DB Account Resources after you're done!](https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/quickstart-portal#clean-up-resources?WT.mc_id=academic-99884-jasmineg)**
 

In [ ]:
# Read the parquet file into Pandas data frame
import pandas as pd
import os
import json
import uuid


filename = 'airports.csv'
print('Reading the csv file into Pandas data frame')
df = pd.read_csv(filename)

#Filter down to
us_airports = df.query('country == "United States"')

us_airports

In [ ]:
#Transform to json
us_airports_json = us_airports.sample(n=3).to_json(orient= 'records') 
us_airports_json

data = json.loads(us_airports_json)
data

In [ ]:

from azure.cosmos import CosmosClient, PartitionKey

ENDPOINT =  os.environ["COSMOS_ENDPOINT"]
KEY = os.environ["COSMOS_KEY"]

DATABASE_NAME = "demo"
CONTAINER_NAME = "airports"

client = CosmosClient(url=ENDPOINT, credential=KEY)

database = client.create_database_if_not_exists(id=DATABASE_NAME)
print("Database\t", database.id)

key_path = PartitionKey(path="/airport_id")

container = database.create_container_if_not_exists(
    id=CONTAINER_NAME, partition_key=key_path, offer_throughput=400
)
print("Container\t", container.id)



for airport in data:
    data[0]['id'] = str(uuid.uuid4())
    container.create_item(data[0])

print('Data has been imported')
